In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA

/home/centos/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
filepath=unicode('/mnt/nas/eeg/processed_data_psd_modified/','utf8')
print filepath
import os
fileName = sorted(os.listdir(filepath))
len(fileName)

/mnt/nas/eeg/processed_data_psd_modified/


29

In [3]:
print fileName

[u'ab.pkl', u'b1.pkl', u'b2.pkl', u'byd.pkl', u'dxb.pkl', u'gyx.pkl', u'hj.pkl', u'hp.pkl', u'ljx.pkl', u'ljy.pkl', u'qxj.pkl', u'shx.pkl', u'sll.pkl', u'szh.pkl', u'wcy.pkl', u'wgp.pkl', u'wj.pkl', u'wls.pkl', u'wqq.pkl', u'wt.pkl', u'wx.pkl', u'wzh.pkl', u'xlz.pkl', u'xsl.pkl', u'xwj.pkl', u'ycl.pkl', u'zgy.pkl', u'zhp.pkl', u'zym.pkl']


In [4]:
def encode1(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 1
        elif label[i] == "N3":
            label[i] = 1
        elif label[i] == "R":
            label[i] = 0
    return label
def encode2(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 0
    return label
def encode3(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 0
    return label
def encode4(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 4
    return label

In [5]:
kf = KFold(n_splits=5, random_state=17, shuffle=True)
for train_index, test_index in kf.split(fileName):
    print("TRAIN:", train_index, "TEST:", test_index)

('TRAIN:', array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 12, 13, 14, 15, 16, 17, 19, 20,
       22, 23, 25, 26, 27, 28]), 'TEST:', array([ 0,  5, 11, 18, 21, 24]))
('TRAIN:', array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       20, 21, 22, 24, 27, 28]), 'TEST:', array([ 2,  3, 19, 23, 25, 26]))
('TRAIN:', array([ 0,  2,  3,  4,  5,  6,  7, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 28]), 'TEST:', array([ 1,  8,  9, 10, 14, 27]))
('TRAIN:', array([ 0,  1,  2,  3,  5,  6,  8,  9, 10, 11, 14, 15, 17, 18, 19, 21, 22,
       23, 24, 25, 26, 27, 28]), 'TEST:', array([ 4,  7, 12, 13, 16, 20]))
('TRAIN:', array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18, 19,
       20, 21, 23, 24, 25, 26, 27]), 'TEST:', array([ 6, 15, 17, 22, 28]))


In [6]:
def create_model():
    model = XGBClassifier(
        learning_rate = 0.008,
        n_estimators = 10000,
        max_depth = 4,
        min_child_weight = 1,
        gamma = 1,
        subsample = 0.7,
        reg_lambda = 0.5,
        colsample_bytree = 0.8,
        objective = 'multi:softmax',
        nthread = -1,
        silent = False,
        seed = 1)
    return model

In [8]:
acc_final = 0
# importance = np.zeros(500,)
filepath1=unicode('/mnt/nas/eeg/processed_data/','utf8')
for train_index, test_index in kf.split(fileName):
    
    trainArr = np.array([])
    testArr = np.array([])
    for i in train_index:
        f = fileName[i]
#         print f
        data = pd.read_pickle(filepath+f)
#         label = pd.read_pickle(filepath1+f).label
#         data = data.join(label)
        train = data#[data.label != '?']
        train = train.replace([np.inf, -np.inf], np.nan)
        train = train.fillna(value = 0)
        train = np.array(train)
        print data.shape
        trainArr = np.append(trainArr, train)

    for i in test_index:
        f = fileName[i]
        #print f
        data = pd.read_pickle(filepath+f)
#         label = pd.read_pickle(filepath1+f).label
#         data = data.join(label)
        test = data#[data.label != '?']
        test = test.replace([np.inf, -np.inf], np.nan)
        test = test.fillna(value = 0)
        test = np.array(test)
        testArr = np.append(testArr, test)
    
    
    trainArr = trainArr.reshape(-1, 9311)
    testArr = testArr.reshape(-1, 9311)
    label_train = trainArr[::,-1]
    label_test = testArr[::,-1]
#     print label_train, label_test
    train = trainArr[::,:9310] 
#     temp = np.concatenate([train,train],axis=1)
#     temp = np.concatenate([temp,train],axis=1)
#     temp[1:,250:500] = train[:-1,::]
#     temp[2:,500:] = train[:-2,::]
#     train = temp
#     del temp
    
    test = testArr[::,:9310]
#     temp = np.concatenate([test,test],axis=1)
#     temp = np.concatenate([temp,test],axis=1)
#     temp[1:,250:500] = test[:-1,::]
#     temp[2:,500:] = test[:-2,::]
#     test = temp
#     del temp

#     dec = PCA(n_components='mle', 
#               copy=True, 
#               whiten=False, 
#               svd_solver='full', 
#               tol=0.0, 
#               iterated_power='auto', 
#               random_state=None)
    dec = SparsePCA(n_components=None, 
                     alpha=1, 
                     ridge_alpha=0.01, 
                     max_iter=1000, 
                     tol=1e-08, 
                     method='lars', 
                     n_jobs=1, 
                     U_init=None, 
                     V_init=None, 
                     verbose=True, 
                     random_state=None)
    
    N_train = encode4(label_train)
    N_test = encode4(label_test)
    train = dec.fit_transform(train)
    test = dec.transform(test)
    print train.shape()
    break
    
    
#     train = train[np.where(N_train != 0)]
#     test = test[np.where(N_test != 0)]
#     label_train = label_train[np.where(N_train != 0)]
#     label_test = label_test[np.where(N_test != 0)]
    
    
    X_train, X_eval, Y_train, Y_eval = train_test_split(
                            train, label_train, test_size=0.1)
    print X_train.shape, X_eval.shape, Y_train.shape, Y_eval.shape
     
#     Y_train = encode2(Y_train)
#     Y_eval = encode2(Y_eval)
#     label_test = encode2(label_test)
    
    print "trianing the model"
    model = create_model()
    model.fit(X_train, Y_train, eval_metric='merror', eval_set=[(X_train, Y_train),(X_eval,Y_eval)], 
              verbose=300,
#               verbose=False,
              early_stopping_rounds = 1000)
    pred = model.predict(test)
    
    acc = pred == label_test
    acc = acc.sum()*1.0 / acc.shape[0]
    print acc
    acc_final += acc
    
    pred = pred.tolist()
    label_test = label_test.tolist()
    a = confusion_matrix(pred, label_test)
    print a
    
#     importance = importance + model.feature_importances_
    del model
print acc_final / 5

(938, 9311)
(1062, 9311)
(1039, 9311)
(1011, 9311)
(1009, 9311)
(978, 9311)
(953, 9311)
(986, 9311)
(1107, 9311)
(896, 9311)
(1025, 9311)
(963, 9311)
(988, 9311)
(1021, 9311)
(972, 9311)
(971, 9311)
(1019, 9311)
(1162, 9311)
(939, 9311)
(920, 9311)
(894, 9311)
(930, 9311)
(887, 9311)
[dict_learning] .++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/centos/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/centos/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/centos/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/centos/anaconda2/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/centos/anaconda2/lib/python2.7/inspect.py", line 1012, in getframeinfo
    lines, lnum = findsource(frame)
  File "/home/centos/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 182, in findsource
    lines = linecache.getlines(file, globals_dict)
  File "


KeyboardInterrupt



In [6]:
from sklearn.ensemble import RandomForestClassifier

In [57]:
def create_model():
    model = RandomForestClassifier(
        n_estimators=1000, 
        max_depth=50, 
        min_samples_split=2, 
        min_samples_leaf=1,
        min_weight_fraction_leaf=0.0, 
        max_features= 30, 
        max_leaf_nodes=None, 
        bootstrap=True, 
        oob_score=True, 
        n_jobs=-1, 
        random_state=17, 
        verbose=1, 
        warm_start=False)
    return model

In [58]:
acc_final = 0
# importance = np.zeros(500,)
filepath1=unicode('/mnt/nas/eeg/processed_data/','utf8')
for train_index, test_index in kf.split(fileName):
    
    trainArr = np.array([])
    testArr = np.array([])
    for i in train_index:
        f = fileName[i]
#         print f
        data = pd.read_pickle(filepath+f)
        label = pd.read_pickle(filepath1+f).label
        data = data.join(label)
        train = data#[data.label != '?']
        train = np.array(train)
        trainArr = np.append(trainArr, train)

    for i in test_index:
        f = fileName[i]
        #print f
        data = pd.read_pickle(filepath+f)
        label = pd.read_pickle(filepath1+f).label
        data = data.join(label)
        test = data#[data.label != '?']
        test = np.array(test)
        testArr = np.append(testArr, test)
    
    
    trainArr = trainArr.reshape(-1, 251)
    testArr = testArr.reshape(-1, 251)
    label_train = trainArr[::,-1]
    label_test = testArr[::,-1]
#     print label_train, label_test
    train = trainArr[::,:250] 
    temp = np.concatenate([train,train],axis=1)
    temp = np.concatenate([temp,train],axis=1)
    temp[1:,250:500] = train[:-1,::]
    temp[2:,500:] = train[:-2,::]
    train = temp
    del temp
    
    test = testArr[::,:250]
    temp = np.concatenate([test,test],axis=1)
    temp = np.concatenate([temp,test],axis=1)
    temp[1:,250:500] = test[:-1,::]
    temp[2:,500:] = test[:-2,::]
    test = temp
    del temp
    
    N_train = encode4(label_train)
    N_test = encode4(label_test)
    
#     train = train[np.where(N_train != 0)]
#     test = test[np.where(N_test != 0)]
#     label_train = label_train[np.where(N_train != 0)]
#     label_test = label_test[np.where(N_test != 0)]
    
    
#     X_train, X_eval, Y_train, Y_eval = train_test_split(
#                             train, label_train, test_size=0.1)
#     print X_train.shape, X_eval.shape, Y_train.shape, Y_eval.shape
     
#     Y_train = encode2(Y_train)
#     Y_eval = encode2(Y_eval)
#     label_test = encode2(label_test)
    
    print "trianing the model"
    model = create_model()
    print label_train.max(),label_train.min()
    model.fit(train, label_train.tolist())
    pred = model.predict(test)
    
    acc = pred == label_test
    acc = acc.sum()*1.0 / acc.shape[0]
    print acc
    acc_final += acc
    
    pred = pred.tolist()
    label_test = label_test.tolist()
    a = confusion_matrix(pred, label_test)
    print a
    
#     importance = importance + model.feature_importances_
    del model
print acc_final / 5

trianing the model
4 0

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   24.1s finished
[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.1s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    0.5s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    0.9s
[Parallel(n_jobs=56)]: Done 1000 out of 1000 | elapsed:    1.4s finished



0.771688741722
[[ 864  104   12    7   67]
 [  35  351   59    0   48]
 [   4  180 1520  138  202]
 [   2   16  320 1411   71]
 [  21   52   41    0  515]]
trianing the model
4 0


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   26.8s finished
[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.1s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    0.4s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    0.8s
[Parallel(n_jobs=56)]: Done 1000 out of 1000 | elapsed:    1.2s finished


0.851044296534
[[1064  137    9    0    4]
 [   9  422   30    0   19]
 [   2  214 1518  103   24]
 [   1    1   32  316    0]
 [   4   36   24    0  388]]
trianing the model
4 

...

[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   24.9s finished
[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.1s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    0.5s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    1.0s
[Parallel(n_jobs=56)]: Done 1000 out of 1000 | elapsed:    1.4s finished


0.76092804304
[[1613   76    7    0    9]
 [ 265  623  181    0  108]
 [  67  289 1448  170   59]
 [   0    0   21  424    0]
 [  30  101   39    0  418]]
trianing the model
4 0


[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 338 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   28.3s finished
[Parallel(n_jobs=56)]: Done  88 tasks      | elapsed:    0.1s
[Parallel(n_jobs=56)]: Done 338 tasks      | elapsed:    0.4s
[Parallel(n_jobs=56)]: Done 688 tasks      | elapsed:    0.8s


0.867009996971
[[ 843    9    3    0   10]
 [  51  304   94    0   59]
 [   6   77 1092   44   14]
 [   1    0   27  274    0]
 [  13   25    6    0  349]]
0.803743887501


[Parallel(n_jobs=56)]: Done 1000 out of 1000 | elapsed:    1.4s finished


In [ ]:
0.830262562699
def create_model():
    model = XGBClassifier(
        learning_rate = 0.008,
        n_estimators = 10000,
        max_depth = 4,
        min_child_weight = 1,
        gamma = 0.5,
        subsample = 0.7,
        reg_lambda = 1,
        colsample_bytree = 0.8,
        objective = 'multi:softmax',
        nthread = -1,
        silent = False,
        seed = 1)
    return model

In [23]:
print acc_final / 5

0.80177797847


In [13]:
importance

array([ 0.06496657,  0.05667299,  0.06310821,  0.06155232,  0.05735025,
        0.05976576,  0.05454363,  0.05513328,  0.05387664,  0.05365849,
        0.04929789,  0.0550574 ,  0.05153917,  0.04909176,  0.04615115,
        0.04384999,  0.05430839,  0.03932109,  0.0415833 ,  0.04679244,
        0.04442161,  0.04927845,  0.03802336,  0.04605909,  0.04929746,
        0.03932587,  0.04210556,  0.04707425,  0.03665189,  0.04849004,
        0.04802434,  0.04436754,  0.04020575,  0.03923525,  0.04833454,
        0.04583556,  0.04852351,  0.04555511,  0.04292247,  0.03891915,
        0.03701295,  0.05052811,  0.04304066,  0.03724106,  0.03838096,
        0.04251988,  0.04077366,  0.04183732,  0.05101721,  0.04018227,
        0.04196293,  0.04161705,  0.04063203,  0.04086308,  0.0437109 ,
        0.04282699,  0.03903559,  0.03635221,  0.04212748,  0.03810578,
        0.04823234,  0.04294663,  0.03938753,  0.03564254,  0.03961834,
        0.03670016,  0.03767151,  0.0363108 ,  0.03618325,  0.03

In [14]:
print importance.shape
print data.columns[-1]

(250,)
label


In [15]:
feature = data.columns[:-1]
zipped = zip(feature,importance)
zipped.sort(key = lambda t:t[1])
zipped.reverse()
im = zipped[:150]

In [16]:
im

[('Chin1-Chin3beta_psd_minOrigin', 0.074360146652907133),
 ('E1-M2_bstd5', 0.064966574776917696),
 ('Snore_bstd5', 0.063108207657933235),
 ('Chin1-Chin3alpha_psd_medianOrigin', 0.062036932911723852),
 ('F3-M2_bstd5', 0.061552324332296848),
 ('ECG1-ECG2Energyrelative4Origin', 0.060652599204331636),
 ('E1-M2Energyrelative5Origin', 0.05976576404646039),
 ('Chin2-Chin3beta_psd_minOrigin', 0.058285326231271029),
 ('RLeg1-RLeg2_bstd5', 0.057350249029695988),
 ('F4-M1delta_psd_medianOrigin', 0.056672992184758186),
 ('F4-M1_bstd5', 0.055133275222033262),
 ('ECG1-ECG2_amin5', 0.055057404097169638),
 ('Chin1-Chin3_astd5', 0.054543632548302412),
 ('O2-M1_bstd5', 0.054308390244841576),
 ('ECG1-ECG2Energyratio2Origin', 0.054118943866342306),
 ('Snore_astd5', 0.053876635618507862),
 ('ECG1-ECG2_kurtosis_14', 0.053799784276634455),
 ('O2-M1Energyrelative5Origin', 0.053658489603549242),
 ('Chin1-Chin3_fftmean1', 0.053371540270745754),
 ('ECG1-ECG2_bmin5', 0.051539169158786535),
 ('Chin1-Chin3_fftmean0

In [17]:
for i in range(150):
    im[i] = list(im[i])


In [18]:
im = np.array(im)

In [19]:
im[0]

array(['Chin1-Chin3beta_psd_minOrigin', '0.0743601466529'], 
      dtype='|S35')

In [20]:
filepath=unicode('/mnt/nas/eeg/processed_data_psd/','utf8')
for i in range(len(fileName)):

    f = fileName[i]
    print f
    data = pd.read_pickle(filepath+f)
    train = data[data.label != '?']
    data = train[im[:,0]]
    print data.shape
    data.to_pickle('/mnt/nas/eeg/processed_data_150_psd/'+fileName[i])

ab.pkl
(1045, 150)
b1.pkl
(938, 150)
b2.pkl
(1062, 150)
byd.pkl
(1018, 150)
dxb.pkl
(1011, 150)
gyx.pkl
(963, 150)
hj.pkl
(148, 150)
hp.pkl
(978, 150)
ljx.pkl
(953, 150)
ljy.pkl
(986, 150)
qxj.pkl
(1033, 150)
shx.pkl
(899, 150)
sll.pkl
(896, 150)
szh.pkl
(1025, 150)
wcy.pkl
(962, 150)
wgp.pkl
(132, 150)
wj.pkl
(1021, 150)
wls.pkl
(972, 150)
wqq.pkl
(1016, 150)
wt.pkl
(7, 150)
wx.pkl
(1017, 150)
wzh.pkl
(1056, 150)
xlz.pkl
(1162, 150)
xsl.pkl
(456, 150)
xwj.pkl
(1061, 150)
ycl.pkl
(920, 150)
zgy.pkl
(894, 150)
zhp.pkl
(918, 150)
zym.pkl
(887, 150)


In [ ]:
10folds:
0.797356084682, data16
model = XGBClassifier(
	learning_rate = 0.008,
	n_estimators = 2000,
	max_depth = 6,
	min_child_weight = 1,
	gamma = 2,
	subsample = 0.7,
	reg_lambda = 0.5,
	colsample_bytree = 0.8,
	objective = 'multi:softmax',
	nthread = -1,
	silent = False,
	seed = 1)

In [ ]:
10folds:
0.793118169349, processed_data15_seperate_energy
model = XGBClassifier(
	learning_rate = 0.008,
	n_estimators = 2000,
	max_depth = 6,
	min_child_weight = 1,
	gamma = 2,
	subsample = 0.7,
	reg_lambda = 0.3,
	colsample_bytree = 0.8,
	objective = 'multi:softmax',
	nthread = -1,
	silent = False,
	seed = 1)

In [ ]:
0.782005495086, data12
model = XGBClassifier(
	learning_rate = 0.01,
	n_estimators = 2000,
	max_depth = 4,
	min_child_weight = 1,
	gamma = 2,
	subsample = 0.7,
	reg_lambda = 0.4,
	colsample_bytree = 0.8,
	objective = 'multi:softmax',
#     objective = 'binary:logistic',
	nthread = -1,
	silent = False,
	seed = 1)